In [35]:
import pandas as pd 
import numpy 

pd.set_option("display.max.columns", 100)
pd.set_option("display.width", 100)
df = pd.read_csv("../data/CMAPSS/train_FD001.txt", 
                 sep=r'\s+', 
                 header=None)
#add the columns
col_names = (
    ["unit_number", "time_in_cycles",
     "operational_setting_1", "operational_setting_2", "operational_setting_3"]
    + [f"sensor_measurement_{i}" for i in range(1, 22)])
df.columns = col_names

df.unit_number.value_counts()
# print(df.head())
# print(df.shape)
# print(df.info())

unit_number
69    362
92    341
96    336
67    313
83    293
     ... 
58    147
70    137
57    137
91    135
39    128
Name: count, Length: 100, dtype: int64